In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
# import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
df = pd.read_csv(r'F:\mpst_full_data.csv', delimiter=',')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 14828 rows and 6 columns


In [4]:
df.head(5)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import preprocessing

Using TensorFlow backend.


In [6]:
df=df.drop(columns=["split"])

In [7]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'synopsis_source'], dtype='object')

In [8]:
df['combined']=df['title']+". "+ df['plot_synopsis']

In [9]:
type(Sequential()) == Sequential


True

In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import plotly.plotly as py
# import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot

In [11]:
def gdd(x):
    a=x.split()
    return len(a)

In [12]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['combined'] = df['combined'].apply(clean_text)

In [13]:
max(df['combined'].apply(gdd))

6119

In [14]:
# label_dict.keys()
x=df['combined']

In [15]:
# tags=label_dict['idx2word']

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [21]:
X = tokenizer.texts_to_sequences(df['combined'].values)
X = pad_sequences(X, maxlen=6119)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (14828, 6119)


In [49]:
# y=pd.get_dummies(df['tags'], )

In [20]:
y = tokenizer.texts_to_sequences(df['tags'].values)
y = pad_sequences(y, maxlen=24)
print('Shape of data tensor:', y.shape)

Shape of data tensor: (14828, 24)


In [19]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=6119, lower=True)
tokenizer.fit_on_texts(df.combined)
sequences = tokenizer.texts_to_sequences(df.combined)
x = pad_sequences(sequences, maxlen=1200)

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df.tags)
y = multilabel_binarizer.classes_

MultiLabelBinarizer(classes=None, sparse_output=False)

In [23]:
# len(label_dict['idx2word'])

In [24]:
x[0]

array([   0,    0,    0, ..., 1464,  840, 5480])

In [25]:
# label_dict.keys()

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
# ths is correct only
# but once you have it done, we need to check the contents of the multilabel_y
multilabel_y = vectorizer.fit_transform(df['tags'])

In [27]:
y = multilabel_y.toarray()

In [28]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=9000)
# x_train=x[:1000]
# x_test=x[1000:]
# y_train=y[:1000]
# y_test=y[1000:]

In [29]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau




In [42]:
timesteps = len(x_train[0])
input_dim = len(x_train[0])
# n_classes = _count_classes(y_train)
n_classes=147
print(timesteps)
print(input_dim)
print(len(x_train[0]))

1200
1200
1200


In [46]:
model = Sequential()
# model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(6119, 71, input_length=1200))
model.add(Dropout(0.15))
model.add(GlobalMaxPool1D())
model.add(Dense(82, activation='sigmoid'))

model.compile(optimizer=Adam(0.015), loss='binary_crossentropy', metrics=['categorical_accuracy'])
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-simple.hdf5', save_best_only=True)
]

history = model.fit(x_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

Train on 10675 samples, validate on 1187 samples
Epoch 1/5
10675/10675 [==============================] - 59s 6ms/step - loss: 0.1489 - categorical_accuracy: 0.1457 - val_loss: 0.1309 - val_categorical_accuracy: 0.1862
Epoch 2/5
10675/10675 [==============================] - 58s 5ms/step - loss: 0.1308 - categorical_accuracy: 0.1799 - val_loss: 0.1284 - val_categorical_accuracy: 0.1896
Epoch 3/5
10675/10675 [==============================] - 59s 5ms/step - loss: 0.1278 - categorical_accuracy: 0.1883 - val_loss: 0.1268 - val_categorical_accuracy: 0.1752
Epoch 4/5
10675/10675 [==============================] - 58s 5ms/step - loss: 0.1259 - categorical_accuracy: 0.1909 - val_loss: 0.1282 - val_categorical_accuracy: 0.1887
Epoch 5/5
10675/10675 [==============================] - 57s 5ms/step - loss: 0.1242 - categorical_accuracy: 0.1904 - val_loss: 0.1289 - val_categorical_accuracy: 0.1946


In [51]:
model = Sequential()
# model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(6119, 71, input_length=1200))
model.add(Dropout(0.50))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.70))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(8, activation='relu'))

model.add(GlobalMaxPool1D())
model.add(Dense(82, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1200, 71)          434449    
_________________________________________________________________
dropout_13 (Dropout)         (None, 1200, 71)          0         
_________________________________________________________________
dense_15 (Dense)             (None, 1200, 256)         18432     
_________________________________________________________________
dropout_14 (Dropout)         (None, 1200, 256)         0         
_________________________________________________________________
dense_16 (Dense)             (None, 1200, 128)         32896     
_________________________________________________________________
dropout_15 (Dropout)         (None, 1200, 128)         0         
_________________________________________________________________
dense_17 (Dense)             (None, 1200, 8)           1032      
__________

In [52]:
model.compile(optimizer=Adam(0.015), loss='binary_crossentropy', metrics=['categorical_accuracy'])
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-simple.hdf5', save_best_only=True)
]



In [ ]:
history = model.fit(x_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

Train on 10675 samples, validate on 1187 samples
Epoch 1/5
 2880/10675 [=======>......................] - ETA: 2:55 - loss: 0.1735 - categorical_accuracy: 0.1187

In [37]:
filter_length = 300
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam


model = Sequential()
model.add(Embedding(6119, 71, input_length=1200))
model.add(Dropout(0.1))
model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(82))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1200, 71)          434449    
_________________________________________________________________
dropout_6 (Dropout)          (None, 1200, 71)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1198, 300)         64200     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 300)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 82)                24682     
_________________________________________________________________
activation_1 (Activation)    (None, 82)                0         
Total params: 523,331
Trainable params: 523,331
Non-trainable params: 0
_________________________________________________________________


In [38]:
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]


In [39]:
history = model.fit(x_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

Train on 10675 samples, validate on 1187 samples
Epoch 1/5
10675/10675 [==============================] - 332s 31ms/step - loss: 0.1692 - categorical_accuracy: 0.1334 - val_loss: 0.1324 - val_categorical_accuracy: 0.1634
Epoch 2/5
10675/10675 [==============================] - 249s 23ms/step - loss: 0.1310 - categorical_accuracy: 0.1647 - val_loss: 0.1298 - val_categorical_accuracy: 0.1778
Epoch 3/5
10675/10675 [==============================] - 249s 23ms/step - loss: 0.1270 - categorical_accuracy: 0.1834 - val_loss: 0.1254 - val_categorical_accuracy: 0.1660
Epoch 4/5
10675/10675 [==============================] - 249s 23ms/step - loss: 0.1226 - categorical_accuracy: 0.1975 - val_loss: 0.1250 - val_categorical_accuracy: 0.1702
Epoch 5/5
10675/10675 [==============================] - 252s 24ms/step - loss: 0.1174 - categorical_accuracy: 0.2128 - val_loss: 0.1244 - val_categorical_accuracy: 0.1752


In [40]:
# filter_length = 300

# model = Sequential()
# model.add(Embedding(6119, 71, input_length=1200))
# model.add(Dropout(0.5))
# model.add(Conv1D(300, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
# model.add(Conv1D(200, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
# model.add(Dense(82))
# model.add(Activation('sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
# model.summary()

In [63]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

max_features = 1024

model = Sequential()
model.add(Embedding(6119, output_dim=71))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(82, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



In [ ]:
# model.fit(x_train, y_train, batch_size=16, epochs=10)
model.fit(x_train, y_train,
          class_weight='balanced',
          epochs=12,
          batch_size=32,
          validation_split=0.2)

Train on 9489 samples, validate on 2373 samples
Epoch 1/12
9489/9489 [==============================] - 715s 75ms/step - loss: 0.1572 - acc: 0.9547 - val_loss: 0.1361 - val_acc: 0.9602
Epoch 2/12
9489/9489 [==============================] - 734s 77ms/step - loss: 0.1392 - acc: 0.9599 - val_loss: 0.1360 - val_acc: 0.9602
Epoch 3/12
9489/9489 [==============================] - 729s 77ms/step - loss: 0.1374 - acc: 0.9600 - val_loss: 0.1360 - val_acc: 0.9602
Epoch 4/12
9489/9489 [==============================] - 579s 61ms/step - loss: 0.1353 - acc: 0.9601 - val_loss: 0.1346 - val_acc: 0.9604
Epoch 5/12
5536/9489 [================>.............] - ETA: 3:00 - loss: 0.1312 - acc: 0.9607

In [ ]:
# score = model.evaluate(x_test, y_test, batch_size=16)

In [ ]:
# model = Sequential()
# # Configuring the parameters
# model.add(LSTM(6119, input_shape=(timesteps, input_dim)))
# # Adding a dropout layer
# model.add(Dropout(0.5))
# # Adding a dense output layer with sigmoid activation
# model.add(Dense(n_classes, activation='sigmoid'))
# model.summary()
# pred=[]
# x= model.predict(y_test)
# for i in x:
#     pred.append(i[:])  
    

In [ ]:
# pred[0].shape

In [ ]:
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_test, pred)

In [ ]:
# model.predict(y_test[0]).shape

In [39]:
# model = Sequential()
# model.add(Embedding(6119, output_dim=71))
# # model.add(LSTM(300))
# model.add(LSTM(256, return_sequences=True))
# model.add(Dropout(0.5))
# model.add(LSTM(128))

# # model.add(LSTM(150))

# # model.add(Dropout(0.5))
# model.add(Dense(82, activation='sigmoid'))

# model.summary()

In [40]:
# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])


In [41]:
# model.fit(x_train, y_train,
#           class_weight='balanced',
#           epochs=12,
#           batch_size=32,
#           validation_split=0.2)